In [1]:
import torch
from datasets import load_dataset, DatasetDict
from llama import BitLlamaConfig, BitLlamaForCausalLM
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer


/Users/ogiwara/Documents/bitnet_b1_58-large/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
ds_name = "range3/wiki40b-ja"
ds_train = load_dataset(ds_name, split="train")
ds_valid = load_dataset(ds_name, split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train,  
        "valid": ds_valid, 
    }
)

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("tokyotech-llm/Swallow-7b-hf")

In [3]:
outputs = tokenizer(
    raw_datasets["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Beginning of train text: 
_START_ARTICLE_
Β-セクレターゼ1
_START_SECTION_
アルツハイマー病における役割
_START_PARAGRAPH_
アルツハイマー病患者の脳で凝集する、40または4
Input IDs length: 15
Input chunk lengths: [128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 105, 128, 128, 128, 72]
Chunk mapping: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [4]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)


In [7]:
config = BitLlamaConfig(
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    hidden_size=768,                  # BitNet論文より
    max_position_embeddings=1024,
    intermediate_size=1536, # Transformerの論文より、途中で拡大する。
    num_attention_heads=12,         # BitNet論文より
    num_hidden_layers=12,            # BitNet論文より
    num_key_value_heads=4,
    torch_dtype=torch.bfloat16,
    rms_norm_eps=1e-06,
)

model = BitLlamaForCausalLM(config)

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [9]:
args = TrainingArguments(
    output_dir="bitnet-127M",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="steps",
    eval_steps=5000,
    logging_steps=2000,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    warmup_steps=5000,
    lr_scheduler_type="linear",  # BitNet論文より
    learning_rate=2.4e-3,  # BitNetb1論文より
    save_steps=2000,
    bf16=True,
    adam_beta1=0.9,
    adam_beta2=0.95,
    save_total_limit=3,
    weight_decay=0.1, # b158
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [8]:
prompt = "昔々あるところに、"
input_ids = tokenizer.encode(
    prompt,
    return_tensors="pt"
)
tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=128,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NotImplementedError: The operator 'aten::isin.Tensor_Tensor_out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.